# Packing CAT Cell demo notebook
The main functions of this demo are located in :
- [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)
- [nlc_optimisation.py](../irregular_object_packing/packing/nlc_optimisation.py)


### Contents
1. 
2. 

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```




In [1]:
# IMPORTS
import numpy as np
import pyvista as pv
import trimesh
from tqdm import tqdm
from importlib import reload

import sys
sys.path.append('../irregular_object_packing/')
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume, translation_matrix
from irregular_object_packing.mesh.utils import print_mesh_info
import irregular_object_packing.packing.chordal_axis_transform as cat
from irregular_object_packing.packing.initialize import create_packed_scene, place_objects, save_image
from irregular_object_packing.packing.plots import create_plot

## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The yellow faces are the CAT faces.

In [2]:
from irregular_object_packing.packing.chordal_axis_transform import main
main()

/opt/homebrew/anaconda3/envs/taichi310/lib/python3.10/site-packages/pyvista/core/pointset.py:309: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)
/opt/homebrew/anaconda3/envs/taichi310/lib/python3.10/site-packages/pyvista/core/pointset.py:318: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)
/opt/homebrew/anaconda3/envs/taichi310/lib/python3.10/site-packages/pyvista/core/pointset.py:327: PyVistaDeprecationWarning: You did not specify a value for `inplac

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Demo computing CAT cells for Blood Cells in a cylindrical container
The cell below shows a demo of computing cat faces for a cylindrical container and multiple blood cell meshes. The yellow faces are the CAT faces and for each red blood cell there is a corresponding CAT cell. 



In [3]:
mesh_volume = 0.1
container_volume = 10
coverage_rate = 0.3

DATA_FOLDER = "../data/mesh/"
loaded_mesh = trimesh.load_mesh(DATA_FOLDER + "RBC_normal.stl")
print_mesh_info(loaded_mesh, "loaded mesh")
# trimesh.Scene([loaded_mesh]).show()
# Scale the mesh to the desired volume
original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)
print_mesh_info(original_mesh, "scaled mesh")

container = trimesh.primitives.Sphere(radius=1, center=[0,0,0])
# container = trimesh.primitives.Cylinder(radius=1, height=1)
print_mesh_info(container, "original container")

container = scale_to_volume(container, container_volume)
print_mesh_info(container, "scaled container")

Mesh info loaded mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, 
volume: 71.04896973139347, 
bounding box: [[35.089 37.963 35.089]
 [42.911 40.037 42.911]] 
center of mass: [39. 39. 39.]

Mesh info scaled mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, 
volume: 0.10000000000000006, 
bounding box: [[-0.4383 -0.1162 -0.4383]
 [ 0.4383  0.1162  0.4383]] 
center of mass: [-0. -0. -0.]

Mesh info original container: <trimesh.primitives.Sphere>, 
volume: 4.1887902047863905, 
bounding box: [[-1. -1. -1.]
 [ 1.  1.  1.]] 
center of mass: [-0.  0.  0.]

Mesh info scaled container: <trimesh.primitives.Sphere>, 
volume: 9.999999999999996, 
bounding box: [[-1.3365 -1.3365 -1.3365]
 [ 1.3365  1.3365  1.3365]] 
center of mass: [0. 0. 0.]



In [4]:
# Initial placement of the objects
objects_coords = place_objects(container, original_mesh, coverage_rate=coverage_rate, c_scale=0.9)


In [5]:
# get vertices of the object meshes and the container
# we resample to simplify and get a more uniform distribution of points
mesh = trimesh.sample.sample_surface_even(original_mesh, 50)[0]
down_scale = 0.1 # initial downscale 
obj_points = []
tf_matrices = []
for i in range(len(objects_coords)):
    # get the object
    object = mesh.copy()
    # random rotation
    M_rot = trimesh.transformations.random_rotation_matrix()
    M_f = trimesh.transformations.scale_matrix(down_scale**(1/3))
    M_t = trimesh.transformations.translation_matrix(objects_coords[i] - np.array([0, 0, 0]))

    M = M_f @ M_rot
    # first scale, then rotate, then translate
    points = trimesh.transform_points(object, M_t @ M)

    tf_matrices.append(M)
    obj_points.append(points)

container_points = trimesh.sample.sample_surface_even(container, 100)[0]

In [6]:
# compute the cat cells for each object (takes approx. 1 min)
import os

stored_data_path = "irop_data.pickle"
# if os.path.exists(stored_data_path):
#     irop_data = cat.IropData.load(stored_data_path)
# else:
irop_data = cat.compute_cat_cells(obj_points, container_points)
irop_data.object_coords = objects_coords
irop_data.save(stored_data_path)


## improvement
transformation optimisation will probably be local to the origin for each mesh. What we can do is use the object_coords to transform the cat cell to the origin, then optimise the transformation, then transform the cat cell back to the object_coords.  

Currently, first it needs to be confirmed that the transformation is local to the origin and that the translation from nlc_optimisation is correct. 

In [7]:
print("ok)")

ok)


In [14]:
import irregular_object_packing.packing.nlc_optimisation as nlc
reload(cat); reload(nlc)
from scipy.optimize import minimize
# for object 0:
    # Define the bounds for the variables
k = 0

def optimal_transform(k, irop_data, scale_bound=(0.1, None), max_angle=1 / 12 * np.pi, max_t=None):
    r_bound = (-max_angle, max_angle)
    t_bound = (0, max_t)
    bounds = [scale_bound, r_bound, r_bound, r_bound, t_bound, t_bound, t_bound]
    x0 = np.array([0.1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

    constraint_dict = {
        "type": "ineq",
        "fun": nlc.constraints_from_dict,
        "args": (
            k,
            irop_data,
        ),
    }
    res = minimize(nlc.objective, x0, method="SLSQP", bounds=bounds, constraints=constraint_dict)
    return res.x

tf_arr = optimal_transform(k, irop_data, scale_bound=(0.1, None), max_angle=1 / 12 * np.pi, max_t=None)


In [15]:
# This next part of code is for the visualization of the CAT cells.
# it processes the output of compute_cat_cells to create a PyVista mesh for each object
# and it applies the corresponding transformations on the object meshes.
from irregular_object_packing.packing.nlc_optimisation import construct_transform_matrix

object_meshes = []
cat_meshes = []
lim = len(irop_data.cat_cells.keys()) 
# for k, v in tqdm(cat_cells.items()):
    # if k >= lim - 1:
    #     break
cat_points, poly_faces = cat.face_coord_to_points_and_faces(irop_data, k)
polydata = pv.PolyData(cat_points, poly_faces)

original_transform_scale_rotate = tf_matrices[k]
old_translation = trimesh.transformations.translation_matrix(objects_coords[k])

computed_transform = construct_transform_matrix(tf_arr, translation=False) 
new_translation = trimesh.transformations.translation_matrix(objects_coords[k] + tf_arr[4:])
modified_transform = new_translation @ computed_transform @ original_transform_scale_rotate

object_mesh = original_mesh.copy()
post_mesh = object_mesh.copy()
object_mesh = object_mesh.apply_transform(old_translation @ original_transform_scale_rotate)
post_mesh = post_mesh.apply_transform(modified_transform)


# post_mesh.vertices = trimesh.transformations.transform_points(
#     post_mesh.vertices, construct_transform_matrix(tf_arr)
# )



In [16]:
# create_plot(objects_coords, object_meshes, cat_meshes, container.to_mesh())
import pyvista as pv

# create the first plot
plotter = pv.Plotter(shape='1|1', notebook=True)  # replace with the filename/path of your first mesh
plotter.subplot(0)
plotter.add_title("Initial Placement")
plotter.add_mesh(object_mesh, color="red", opacity=0.8)
plotter.add_mesh(polydata, color="yellow", opacity=0.4)

# create the second plot
# plot2 = pv.Plotter()
plotter.subplot(1)
plotter.add_title("Optimized Placement")
plotter.add_mesh(post_mesh, color="red", opacity=0.8)
plotter.add_mesh(polydata, color="yellow", opacity=0.4)

# set up the rendering window with two horizontal subplots
# plotter.add_subplot(plot1, render=False)
# plotter.add_subplot(plot2, render=False)

# show the plot
plotter.show()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)